In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipelin import PipelinePromptTempalte

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

prompt.format(country="Germany")

'What is the capital of Germany'

### ChatOpenAI 초기화
- **Temperature**: 출력의 창의성을 조정합니다.
- **Streaming**: `streaming=True`로 설정하면 응답이 스트리밍 방식으로 생성됩니다.
- **StreamingStdOutCallbackHandler**: 스트리밍된 데이터를 콘솔에 출력합니다.

### PromptTemplate 생성
- **Prompt Templates**: `intro`, `example`, `start`와 같은 템플릿은 프롬프트의 서로 다른 부분을 정의합니다:
  - **`intro`**: 역할을 지정합니다 (예: "해적").
  - **`example`**: 대화 스타일의 예시를 제공합니다.
  - **`start`**: 대화의 시작 질문을 정의합니다.

### PipelinePromptTemplate
- 여러 템플릿을 결합하여 하나의 완성된 대화 프롬프트를 생성합니다.

### 체인 실행
- `chain.invoke()`를 통해 입력 데이터를 기반으로 최종 텍스트를 생성합니다.

### 예시
당신은 "해적"이라는 역할을 부여받았으며, 예제와 실제 질문을 포함한 대화가 출력됩니다.


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature=0.1,  # 모델 출력의 창의성 제어. 낮을수록 더 결정적인 답변을 생성
    streaming=True,   # 스트리밍 방식으로 응답을 생성 (단어 단위로 실시간 출력)
    callbacks=[
        StreamingStdOutCallbackHandler(),  # 스트리밍된 데이터를 콘솔에 출력
    ],
)

# **Prompt 정의**

# 'intro' 프롬프트 템플릿: 역할을 부여하는 텍스트 생성
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

# 'example' 프롬프트 템플릿: 대화 스타일의 예시를 제공
example = PromptTemplate.from_template(
    """
    This is an example of how you talk: 
    Human: {example_question}
    You: {example_answer}
    """
)

# 'start' 프롬프트 템플릿: 대화의 시작 부분을 정의
start = PromptTemplate.from_template(
    """
    Start now! 
    Human: {question}
    You:
    """
)

# 최종 프롬프트 템플릿: 위의 템플릿들을 결합하여 최종 대화 형식 생성
final = PromptTemplate.from_template(
    """
    {intro}                                          
    {example}                                    
    {start}
    """
)

# 파이프라인에서 사용할 각 프롬프트를 정의 (이름과 템플릿 매칭)
prompts = [
    ("intro", intro),      # 역할 부여 템플릿
    ("example", example),  # 대화 예제 템플릿
    ("start", start),      # 대화 시작 템플릿
]

# **PipelinePromptTemplate**
# 여러 개의 프롬프트 템플릿을 결합하여 최종 대화 프롬프트를 생성
full_prompt = PipelinePromptTemplate(
    final_prompt=final,  # 최종적으로 사용할 프롬프트
    pipeline_prompts=prompts,  # 위에서 정의한 개별 템플릿
)

# **체인 생성**
# 파이프라인 프롬프트를 ChatOpenAI 모델과 연결
chain = full_prompt | chat

# **체인 실행**
# 사용자가 제공한 입력 데이터를 기반으로 체인 실행
chain.invoke(
    {
        "character": "Pirate",  # 역할: 해적
        "example_question": "What is your location?",  # 예제 질문
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",  # 예제 답변
        "question": "What is your fav food?",  # 실제 질문
    }
)


Arrr matey! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, it be the perfect meal for a pirate like meself! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, it be the perfect meal for a pirate like meself! Arrr!")